# elixir 函数式编程语言

## Financial Team TWIL Shared

> Shared by [laserx](https://github.com/laserx)

## 参考

* [A week with Elixir](https://joearms.github.io/#2013-05-31%20A%20week%20with%20Elixir)
* [Why You Too Should Learn Elixir](https://mixandgo.com/learn/why-you-too-should-learn-elixir)
* [Comparing Elixir and Erlang variables](http://blog.plataformatec.com.br/2016/01/comparing-elixir-and-erlang-variables/)
* [github: ovrmrw/elixir_file_read_eager_lazy_flow](https://github.com/ovrmrw/elixir_file_read_eager_lazy_flow)
* [Measuring And Visualizing Genstage/flow With Gnuplot](http://teamon.eu/2016/measuring-visualizing-genstage-flow-with-gnuplot/)
* [How Discord Handles Two and Half Million Concurrent Voice Users using WebRTC](https://blog.discordapp.com/how-discord-handles-two-and-half-million-concurrent-voice-users-using-webrtc-ce01c3187429)
* [How Discord Scaled Elixir to 5,000,000 Concurrent Users](https://blog.discordapp.com/scaling-elixir-f9b8e1e7c29b)

---

In [1]:
"MEASURING AND VISUALIZING GENSTAGE/FLOW WITH GNUPLOT"
|> String.split(" ")
|> Enum.map(&String.capitalize/1)
|> Enum.join(" ")

"Measuring And Visualizing Genstage/flow With Gnuplot"

## 背景

elixir 是一门利用 erlang VM (beam) 开发的动态函数, 兼具了 erlang 的大量特性, 比如并发能力, 容错, 热更新, 分布式特性等.

个人接触 elixir 的时间大约是在16年左右, 当时对 elixir 完全没有了解的前提下, 参加了一个线下分享活动, 当时的分享者着重分享了 elixir 中的 macro. 当时浅尝即止, 没有做过多的深入探索. 当17年的时候, 接触到了 elixir 的一个库, flow, 主要用于进行单机并行计算. 在此之后, 重燃对 elixir 的兴趣.

erlang 作为存在了30余年的语言, 语言学习曲线陡峭, 语法与主流语言差异较大, 劝退大量开发者, 但其优秀的语言特性, erlang/OTP 整体设计的质量等等优点让这门语言散发着光芒. elixir 出现弥补了部分语法的困扰.

elixir 语言的作者是 [José Valim](https://github.com/josevalim), 原 Rails core team 成员, 所以, elixir 在语法上有着非常明显的 ruby 语法的痕迹, 但除此以外, 其和 ruby 基本没有什么过多的相似性.

---

## 基本概念

### 命令式编程和函数编程

imperative 和 functional 这两种编程范式存在已久, 我们接触的大多数编程语言都是命令式编程范畴的语言, 纯函数式语言主要为 haskell, 而 elixir 或者说 erlang 并不是纯函数式语言, 我们平时谈论的 clojure, scala 等, 大多不能称之为纯函数语言.

### 不可变性与模式匹配

immutable 是大多数函数式语言的基本特性, 在 erlang 中变量 X 被创建后, 其的值不可更改. 

```erlang
1> X = 1.
1
2> X = 2.
** exception error: no match of right hand side value 2
```

而在 elixir 中:

In [2]:
a = 1
a = "rebinding"

"rebinding"

在 elixir 中, `=` 不是赋值操作符, 而是匹配操作符. 其本质是操作符左侧变量与右侧变量匹配, 匹配成功, 值与变量绑定, 反之抛出异常. 

让我们看一下示例:

In [3]:
[1, a] = [1, 2]

[1, 2]

In [4]:
a

2

上例中, 左侧 `[1, a]` 中的 `1` 匹配右侧 `[1, 2]` 中的 `1`, 匹配成功. `a` 匹配 `2`, 进行绑定, 绑定后, 变量 a 的值为 2.

接下来:

In [5]:
[2, a] = [1, 2]

MatchError: 1

其中, 左侧 `[2, a]` 中的 `2` 匹配右侧 `[1, 2]` 中的 `1`, 匹配失败, 抛出异常.

我们继续执行匹配:

In [5]:
[2, a] = [2, 3]

[2, 3]

In [6]:
a

3

此时, 匹配成功, 变量 a 绑定为 3.

当我们需要使用变量的值进行匹配时, 我需要使用 `^`, 称之为 pin 操作符.

下例中, 变量 a 绑定的值依然是 3:

In [7]:
[2, ^a] = [2, 1]

MatchError: 1

在此, 3 匹配 1 失败, 抛出异常.

elixir 中对变量的操作与 erlang 的主要不同是, elixir 将变量值匹配使用显性的 pin 操作符表达, 替代了 erlang 中的隐式匹配模式, 在 [Comparing Elixir and Erlang variables](http://blog.plataformatec.com.br/2016/01/comparing-elixir-and-erlang-variables/) 这篇 José 的博客中详细的进行了展开.

到此为止, 我们了解了模式匹配和绑定以及重绑定, 那是不是说, elixir 变量是可变的呢?

而事实是, elixir 中没有可变变量. 不可变性的本质是不允许特定内存地址的值被修改, 直到其被垃圾回收或者超出作用域.

而绑定的行为更像是为值本身打了一个标签, 标签是可以重新打在别的值上, 而值本身不可以本改变.

这样, 也就导致了我们无法在 elixir 中使用 for 循环, 而实现 for 循环, 只能使用推导式或者递归.


### 递归

递归在 elixir 使用的频次可能没有那么的高, 因为有 `Enum` 和 `Stream` 模块, 提供了相关的 `map` 和 `reduce` 方法.

而有的时候使用递归也是个不错的选择, 下面是一段示例:

In [7]:
defmodule Recursion do
  def double(enumerable, acc \\ [])
  def double([], acc), do: acc

  def double([h | t], acc) do
    double(t, acc ++ [h * 2])
  end
end


{:module, Recursion, <<70, 79, 82, 49, 0, 0, 5, 76, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 135, 0, 0, 0, 15, 16, 69, 108, 105, 120, 105, 114, 46, 82, 101, 99, 117, 114, 115, 105, 111, 110, 8, 95, 95, 105, 110, 102, ...>>, {:double, 2}}

In [8]:
Recursion.double([1, 20, 12, 34])

[2, 40, 24, 68]

在段代码中, 定义了一个模块 `Recursion`, 其中提供了3个函数, 而三个函数具有同样的函数名称及元数, 通过模式匹配, 将列表中的所有元素乘以 2, 并返回结果列表, 同时, elixir 具备尾递归优化能力.

## 为什么可以学习 elixir 语言

这一小部分可能不够客观.

首先, 学不学 elixir 语言本身并不重要, 主要是可以通过学习本身了解不同编程范式之间的差异, 以及对同样问题不同的解.

elixir 在所有编程语言绝对是生态相对较小的一族, 至今(2019年初)发布至 [hex.pm](https://hex.pm/) 中的包只有不到8000个, 这其中相对不错或者说相对活跃的包总数并不多, 但是这并不应该阻碍我们去学习函数式编程语言. 例如 rust, 应该算是近2年的热门语言, 其就是混合范式语言, 包含了的不可变性及模式匹配的特性, 所以至少了解一门函数式编程语言并没有什么坏处, 而这其中的选择又是那么的多, haskell, clojure, racket, elm ......